In [2]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')
plt.rcParams["figure.figsize"] = (10, 4)

In [3]:
!curl -L -O https://raw.githubusercontent.com/github/innovationgraph/main/data/topics.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  820k  100  820k    0     0  3101k      0 --:--:-- --:--:-- --:--:-- 3269k


In [4]:
df = pd.read_csv("topics.csv")

In [23]:
df_by_topic = df.groupby(["iso2_code", "topic"], as_index=False).sum()[["iso2_code", "topic", "num_pushers"]]

In [25]:
df_by_topic[df_by_topic["iso2_code"] == "US"]

,iso2_code,topic,num_pushers
3573,US,3d,2990
3574,US,3d-graphics,1012
3575,US,3d-printing,767
3576,US,a11y,1336
3577,US,accessibility,2942
...,...,...,...
4528,US,yaml,2590
4529,US,yarn,314
4530,US,youtube,2711
4531,US,zsh,5774


In [28]:
df[(df["iso2_code"] == "US") & (df["topic"] == "youtube")]["num_pushers"].sum()

2711

In [42]:
df_pushers_by_iso2_code = df.groupby("iso2_code").sum()[["num_pushers"]]

In [43]:
df_pushers_by_iso2_code.loc["AE"]

num_pushers    13172
Name: AE, dtype: int64

In [44]:
df_by_topic["percent"] = df_by_topic.apply(
    lambda x: x["num_pushers"] / df_pushers_by_iso2_code.loc[x["iso2_code"]],
    axis=1
)

In [49]:
df_by_topic[df_by_topic["iso2_code"] == "US"].sort_values("percent", ascending=False).head(25)

,iso2_code,topic,num_pushers,percent
3742,US,config,303932,0.080112
3938,US,github-config,301620,0.079503
4254,US,python,107278,0.028277
4027,US,javascript,76327,0.020119
4269,US,react,66462,0.017519
3966,US,hacktoberfest,56740,0.014956
4024,US,java,41875,0.011038
4451,US,typescript,40475,0.010669
4080,US,machine-learning,39723,0.010470
4161,US,nodejs,39406,0.010387


In [52]:
df_by_topic.groupby("topic").sum("percent").sort_values("percent", ascending=False).head(40)

,num_pushers,percent
topic,,
config,2395165,52.098495
github-config,2383251,51.957808
python,562889,3.262977
javascript,491918,3.083119
react,402560,2.494107
hacktoberfest,309417,1.632929
typescript,269877,1.381788
java,275705,1.318244
nodejs,243211,0.905262


In [72]:
top40 = df_by_topic.groupby("topic").sum("percent").sort_values("percent", ascending=False).head(42).tail(40).index.values

In [73]:
top40

array(['python', 'javascript', 'react', 'hacktoberfest', 'typescript',
       'java', 'nodejs', 'css', 'docker', 'html', 'reactjs', 'php',
       'machine-learning', 'golang', 'api', 'android', 'python3', 'vue',
       'linux', 'kubernetes', 'go', 'rust', 'deep-learning', 'blog',
       'nextjs', 'website', 'html5', 'dotfiles', 'cpp', 'mongodb', 'game',
       'cli', 'django', 'mysql', 'documentation', 'csharp', 'kotlin',
       'tailwindcss', 'ios', 'blockchain'], dtype=object)

In [74]:
df_iso2_top40_topics = df_by_topic.pivot(index="iso2_code", columns="topic", values="percent")[top40]

In [75]:
df_iso2_top40_topics.fillna(0.0, inplace=True)

In [92]:
df_iso2_top40_topics = df_iso2_top40_topics[df_iso2_top40_topics.sum(axis=1) > 0]

In [97]:
df_iso2_top40_topics.corr()["api"].sort_values(ascending=False)

topic
api                 1.000000
linux               0.767505
game                0.755214
python3             0.744936
php                 0.739617
website             0.717155
docker              0.635987
cli                 0.631026
documentation       0.619451
android             0.591221
cpp                 0.578590
machine-learning    0.560476
rust                0.552688
csharp              0.520802
dotfiles            0.490439
mongodb             0.484610
blockchain          0.478675
html                0.478128
nodejs              0.441061
kotlin              0.433229
go                  0.423809
nextjs              0.406515
golang              0.402480
django              0.386180
css                 0.370722
ios                 0.362359
reactjs             0.352879
tailwindcss         0.329071
kubernetes          0.325339
deep-learning       0.324722
typescript          0.263458
java                0.249967
mysql               0.223006
html5               0.214974
vue     

In [121]:
df_iso2_top40_topics.T.corr()["DK"].sort_values(ascending=False)

iso2_code
DK    1.000000
BE    0.867652
IE    0.857226
NZ    0.829844
AT    0.815126
        ...   
VE    0.051029
GH    0.051029
UZ    0.051029
BY    0.038756
PE    0.022740
Name: DK, Length: 64, dtype: float64

In [123]:
with open("github-topics-by-economy.csv", "w") as fh:
    fh.write(df_iso2_top40_topics.to_csv())